In [146]:
import PyGnuplot as gp
import numpy as np
import json

In [147]:
with open("input.json", "r") as read_file:
    data = json.load(read_file)
data

{'world': {'size': {'N': 4, 'M': 4},
  'states': {'S': [1, 1], 'T': [[4, 1]], 'F': [[3, 1]], 'B': [[3, 2]]}},
 'transition_rates': {'p1': 0.8, 'p2': 0.1, 'p3': 0.1},
 'reward_function': {'r': -1, 'r_T': [100], 'r_B': [-20]},
 'gamma': 0.99}

In [148]:
class MarkovDecisionProcess:
    def __init__(self, *, N, M, S, T, F, B, p1, p2, p3, r, r_T, r_B, gamma):
        self.N, self.M = N, M
        self.S = S
        self.T = self.lists_to_tuples(T)
        self.F = self.lists_to_tuples(F)
        self.B = self.lists_to_tuples(B)
        self.p1, self.p2, self.p3 = p1, p2, p3
        self.r, self.r_T, self.r_B = r, r_T, r_B
        self.gamma = gamma
        
    def lists_to_tuples(self, I):
        return [(a, b) for a, b in I]

mdp = MarkovDecisionProcess(**data['world']['size'],
                            **data['world']['states'],
                            **data['transition_rates'],
                            **data['reward_function'],
                            gamma=data['gamma'])

In [149]:
states = {(x,y) for x in range(1, mdp.N + 1) 
                for y in range(1, mdp.M + 1) if (x, y) not in mdp.F}

rewards = {(x, y): mdp.r for (x, y) in states}
rewards.update({(x, y): terminal_r for (x, y), terminal_r in zip(mdp.T, mdp.r_T)})
rewards.update({(x, y):    bonus_r for (x, y), bonus_r    in zip(mdp.B, mdp.r_B)})

In [150]:
def invalid_state(x, y):
    out_of_world = x < 1 or x > mdp.N or y < 1 or y > mdp.M
    return (x, y) in mdp.F or out_of_world

def generate_transitions(states):
    probabilities = [mdp.p1, mdp.p2, mdp.p3, 1 - (mdp.p1 + mdp.p2 + mdp.p3)]
    transitions = dict()
    for x, y in states:
        src_state = (x, y)
        for move in ['U', 'L', 'R', 'D']:
            if move == 'U':
                destination = (x, y + 1)
                left = (x - 1, y)
                right = (x + 1, y)
                opposite = (x, y - 1)

            if move == 'L':
                destination = (x - 1, y)
                left = (x, y - 1)
                right = (x, y + 1)
                opposite = (x + 1, y)

            if move == 'D':
                destination = (x, y - 1)
                left = (x + 1, y)
                right = (x - 1, y)
                opposite = (x, y + 1)

            if move == 'R':
                destination = (x + 1, y)
                left = (x, y + 1)
                right = (x, y - 1)
                opposite = (x - 1, y)

            for dst_state, transition_rate in zip([destination, left, right, opposite], probabilities):
                if invalid_state(*dst_state):
                    dst_state = x, y
                
                if src_state in transitions:
                    if move in transitions[src_state]:
                        transitions[src_state][move].append((dst_state, transition_rate))
                    else:
                        transitions[src_state][move] = [(dst_state, transition_rate)]
                else:
                    transitions[src_state] = {move: [(dst_state, transition_rate)]}
    return transitions

transitions = generate_transitions(states)
transitions.update({(x, y): {'END': [((x, y), 0)]} for x, y in mdp.T})

In [151]:
class ValueIteration:
    def __init__(self, states, rewards, transitions, stop_cond=0.0001):
        self.states = states
        self.rewards = rewards
        self.transitions = transitions
        self.stop_cond = stop_cond
        self.t = 0  # liczba iteracji
        self.converged = False
        
    def R(self, state):
        return self.rewards[state]
    
    def T(self, state, action):
        return self.transitions[state][action]
    
    def actions(self, state):
        return self.transitions[state].keys()
        
    def iterate(self):
        utility_history = []
        U1 = {s: 0 for s in states}
        while not self.converged:
            self.converged = True
            self.t += 1
            U = U1.copy()
            utility_history.append(U)
            delta = 0
            for s in self.states:
                U1[s] = self.R(s) + mdp.gamma * max([sum([p * U[s1] for (s1, p) in self.T(s, a)]) 
                                                     for a in self.actions(s)])
                delta = max(delta, abs(U1[s] - U[s]))
                
            if delta >= self.stop_cond:
                self.converged = False
        return U, utility_history
    
    def best_policy(self, U):
        pi = {}
        for s in self.states:
            pi[s] = max(self.actions(s), key=lambda a: self.expected_utility(a, s, U))
        return pi
            
    def expected_utility(self, a, s, U):
        return sum([p * U[s1] for (s1, p) in self.T(s, a)])

vi = ValueIteration(states, rewards, transitions)
u, history = vi.iterate()
print("Liczba iteracji: ", vi.t)
pi = vi.best_policy(u)

Liczba iteracji:  32


In [152]:
for j in reversed(range(1, mdp.M + 1)):
    for i in range(1, mdp.N + 1):
        print("{:.4f}".format(u.get((i, j), 0.)), end=' ');
    print(end='\n')

print()

81.9383 84.2609 86.5860 88.8827 
81.7354 84.2724 87.0596 91.5547 
79.5935 80.5997 70.4670 94.5352 
77.4524 78.2494 0.0000 100.0000 



In [153]:
arrows = {'U': '^', 'L': '<', 'R': '>', 'D': 'v'}

for j in reversed(range(1, mdp.M + 1)):
    for i in range(1, mdp.N + 1):
        print("{}".format(arrows.get(pi.get((i, j), ' '), ' ')), end=' ');
    print(end='\n')

print()

> > > v 
> > > v 
^ ^ > v 
^ ^     



In [154]:
# zmiana słownika 'history' do postaci użytecznej dla tworzenia kolumn dla każdego stanu
state_history = {}
for u in history:
    for s in u:
        if s in state_history:
            state_history[s].append(u[s])
        else:
            state_history[s] = [u[s]]
state_history

{(1, 2): [0,
  -1.0,
  -1.99,
  -2.9701000000000004,
  -3.9403990000000007,
  20.806110302768015,
  27.81122827205111,
  46.33487575030555,
  51.871987920031245,
  59.698640926992965,
  64.63891186367717,
  70.18526235053372,
  73.52720498939073,
  76.06768738672227,
  77.46783941872252,
  78.3897130518767,
  78.89432106575815,
  79.20266918978436,
  79.37153186156893,
  79.470372718803,
  79.52455768980683,
  79.5554633698469,
  79.57241351682643,
  79.58192880495915,
  79.58714682016006,
  79.59004716477331,
  79.59163694915459,
  79.59251511506257,
  79.59299619185487,
  79.59326088249624,
  79.59340580120029,
  79.59348533562208],
 (3, 2): [0,
  -20.0,
  -20.99,
  38.01281200000002,
  49.726339936,
  61.25946988192,
  64.87452165324774,
  67.74748052025413,
  68.8744467182388,
  69.64805671664928,
  70.0018655676847,
  70.22074138024671,
  70.32960807054766,
  70.39317525561918,
  70.42622476650872,
  70.44493039328525,
  70.45488526414314,
  70.46042662485945,
  70.46341216633985,

In [155]:
# tworzenie listy kolumn oraz listy z kolejnością występowania stanów, tę drugą wykorzystujemy przy tworzeniu komendy dla gnuplota
columns_to_concatenate = []
state_order = []

for s in state_history:
    columns_to_concatenate.append(np.array(state_history[s])[:, np.newaxis])
    state_order.append(s)
    
# sklejenie kolumn wzdłuż drugiej współrzędnej (axis=1) i doklejenie numerów iteracji jako pierwszej kolumny
A = np.concatenate(columns_to_concatenate, axis=1)
A = np.concatenate((np.arange(len(history))[:, np.newaxis], A), axis=1).T

In [156]:
# zdefiniowanie nazwy pliku i zapisanie tabeli do pliku
filename = "value_iteration.dat"
gp.s(A, filename)

In [158]:
# tworzenie komendy poprzez doklejanie stringów rysujących kolejne linie
command = 'plot '
for i, state in enumerate(state_order):
    command += "'{}' using 1:{} title '{}' with lines, ".format(filename, i+2, state)

# rysowanie wykresu
gp.c(command)

# zapisywanie komendy do pliku
with open('command.txt', 'w') as f:
    f.write(command)